In [2]:
import torch
import torch.nn as nn # 신경망들이 포함됨
import torch.optim as optim # 최적화 알고리즘들이 포함됨
import torch.nn.init as init # 텐서에 초기값을 줌

import torchvision.datasets as datasets # 이미지 데이터셋 집합체
import torchvision.transforms as transforms # 이미지 변환 툴

from torch.utils.data import DataLoader # 학습 및 배치로 모델에 넣어주기 위한 툴

import numpy as np
import matplotlib.pyplot as plt

In [5]:
batch_size = 100
learning_rate = 0.0002
num_epoch = 10

In [3]:
mnist_train = datasets.MNIST(root="../Data/", train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
mnist_test = datasets.MNIST(root="../Data/", train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

100.0%


Extracting ../Data/MNIST\raw\train-images-idx3-ubyte.gz to ../Data/MNIST\raw



100.0%


Extracting ../Data/MNIST\raw\train-labels-idx1-ubyte.gz to ../Data/MNIST\raw



100.0%


Extracting ../Data/MNIST\raw\t10k-images-idx3-ubyte.gz to ../Data/MNIST\raw



100.0%

Extracting ../Data/MNIST\raw\t10k-labels-idx1-ubyte.gz to ../Data/MNIST\raw



In [6]:
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=True)

In [7]:
class CNN(nn.Module):
    def __init__(self):
        # super함수는 CNN class의 부모 class인 nn.Module을 초기화
        super(CNN, self).__init__()
        
        # batch_size = 100
        self.layer = nn.Sequential(
            # [100,1,28,28] -> [100,16,24,24]
            nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5),  # 1채널 입력을 16채널로 변환하는 5x5 컨볼루션 레이어
            nn.ReLU(),  # ReLU 활성화 함수
            
            # [100,16,24,24] -> [100,32,20,20]
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5),  # 16채널 입력을 32채널로 변환하는 5x5 컨볼루션 레이어
            nn.ReLU(),  # ReLU 활성화 함수
            
            # [100,32,20,20] -> [100,32,10,10]
            nn.MaxPool2d(kernel_size=2,stride=2),  # 2x2 맥스 풀링 레이어
            
            # [100,32,10,10] -> [100,64,6,6]
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5),  # 32채널 입력을 64채널로 변환하는 5x5 컨볼루션 레이어
            nn.ReLU(),  # ReLU 활성화 함수
            
            # [100,64,6,6] -> [100,64,3,3]
            nn.MaxPool2d(kernel_size=2,stride=2)  # 2x2 맥스 풀링 레이어          
        )
        self.fc_layer = nn.Sequential(
            # [100,64*3*3] -> [100,100]
            nn.Linear(64*3*3,100),  # 64*3*3 차원의 입력을 100차원으로 변환하는 선형 레이어                                              
            nn.ReLU(),  # ReLU 활성화 함수
            # [100,100] -> [100,10]
            nn.Linear(100,10)  # 100차원의 입력을 10차원으로 변환하는 선형 레이어                                                   
        )       
        
    def forward(self,x):
        # self.layer에 정의한 연산 수행
        out = self.layer(x)
        # view 함수를 이용해 텐서의 형태를 [100,나머지]로 변환
        out = out.view(batch_size,-1)
        # self.fc_layer 정의한 연산 수행    
        out = self.fc_layer(out)
        return out

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [9]:
model = CNN().to(device)

In [10]:
loss_func = nn.CrossEntropyLoss()
#손실함수는 cross entropy loss를 사용

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
# 최적화 알고리즘은 Adam을 사용

In [11]:
# 손실에대한 경사하강법을 진행하여 모델을 업데이트
loss_arr =[]

# num_epoch 만큼 반복
for i in range(num_epoch):
    # train_loader에서 이미지와 레이블을 가져옴
    for j,[image,label] in enumerate(train_loader):
        x = image.to(device)
        y= label.to(device)
        
        optimizer.zero_grad() # 기울기를 0으로 초기화
        
        # 모델에 이미지를 전달하여 출력을 얻음
        output = model.forward(x)
        
        # 출력과 레이블을 사용하여 손실 계산
        loss = loss_func(output,y)
        loss.backward() #기울기 계산
        optimizer.step() # 가중치 업데이트
        
        # 매 1000번째 반복마다 손실 출력 및 loss_arr에 손실 값 추가
        if j % 1000 == 0:
            print(loss)
            loss_arr.append(loss.cpu().detach().numpy())  # detach() 함수는 기존 텐서에서 계산된 텐서를 분리시킴

tensor(2.2968, grad_fn=<NllLossBackward0>)
tensor(0.1856, grad_fn=<NllLossBackward0>)
tensor(0.0583, grad_fn=<NllLossBackward0>)
tensor(0.0896, grad_fn=<NllLossBackward0>)
tensor(0.0232, grad_fn=<NllLossBackward0>)
tensor(0.0602, grad_fn=<NllLossBackward0>)
tensor(0.0151, grad_fn=<NllLossBackward0>)
tensor(0.0651, grad_fn=<NllLossBackward0>)
tensor(0.0166, grad_fn=<NllLossBackward0>)
tensor(0.0169, grad_fn=<NllLossBackward0>)


In [12]:
correct = 0
total = 0

# evaluate model
model.eval()

with torch.no_grad():
    for image,label in test_loader:
        x = image.to(device)  # 이미지를 디바이스로 전송
        y= label.to(device)  # 라벨을 디바이스로 전송

        output = model.forward(x)  # 모델에 이미지를 전달하여 출력값 도출
        
        # torch.max함수는 (최댓값,index)를 반환 
        _,output_index = torch.max(output,1)  # 출력값에서 최댓값과 해당 인덱스를 추출
        
        # 전체 개수 += 라벨의 개수
        total += label.size(0)  # 전체 개수에 현재 배치의 라벨 개수를 더함
        
        # 도출한 모델의 index와 라벨이 일치하면 correct에 개수 추가
        correct += (output_index == y).sum().float()  # 모델의 예측과 실제 라벨이 일치하는 개수를 correct에 더함
    
    # 정확도 도출
    print("Accuracy of Test Data: {}%".format(100*correct/total))  # 정확도를 계산하여 출력

Accuracy of Test Data: 99.11000061035156%
